In [ ]:
# 기사 목록

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# 검색어, 시작 날짜, 종료 날짜 입력
keywords = [
    "검색어",
]
start_date = "2024-06-01"
end_date = "2024-08-31"
seen_urls = set()

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

# 검색 결과 가져오는 함수
def fetch_search_results(keyword, start_date, end_date):
    search_url = "https://www.kmib.co.kr/index.asp"
    driver.get(search_url)

    # 날짜 문자열을 datetime 객체로 변환하는 함수
    def parse_date(date_str):
        try:
            return datetime.strptime(date_str, "%Y-%m-%d")  # "YYYY-MM-DD" 형식 변환
        except ValueError:
            return None  # 변환 실패 시 None 반환

    # 날짜 범위 필터를 위한 datetime 변환
    start_date_str = datetime.strptime(start_date, "%Y-%m-%d")
    end_date_str = datetime.strptime(end_date, "%Y-%m-%d")   

    results = []
    page = 1
     

    # 검색어 입력 및 검색 버튼 클릭
    try:
        # 검색 버튼 찾기 (검색 창 열기용)
        search = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, 'srchBtn'))
        )
        search.click()  # 검색 창 열기

        # 검색어 입력
        search_box = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, 'search'))
        )
        search_box.clear()  # 기존 텍스트 제거
        search_box.send_keys(keyword)  # 키워드 입력

        # 검색 실행 버튼 찾기 및 클릭
        search_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.btn_search'))
        )
        search_button.click()  # 클릭

        # 검색 결과가 나타날 때까지 대기
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.ID, 'searchList'))
        )
        time.sleep(2)  # 페이지 로드 대기
    except Exception as e:
        print(f"검색 실행 중 오류 발생: {e}")
        return []
    
    while True:
        print(f"{keyword} - 페이지 {page}")

        # 페이지 내용을 파싱
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        articles = soup.find("div", id="searchList")

        if not articles:
            print(f"{keyword}에 대한 더 이상의 기사가 없습니다. 페이지: {page}")
            return

        # 기사 정보 추출
        items = articles.find_all("div", class_="card_body rgap_lg1")
        for item in items:
            try:
                title_tag = item.find("div", class_="tit fs_lg20 fs_xs16 fw_lgmd lh_lg15").find("a")
                if title_tag:
                    title = title_tag.text.strip()
                    url = title_tag["href"]

                    if url in seen_urls:
                        continue  # 중복된 URL 무시

                    seen_urls.add(url)

                    date_element = item.find("div", class_="date gray500 fs_lg14 fw_lgrg")
                    date_str = date_element.text.strip() if date_element else ""
                    article_date = parse_date(date_str)

                    # **start_date 이전 기사가 나오면 루프 종료**
                    if article_date and article_date < start_date_str:
                        print(f"{keyword}에 대한 기사가 시작 날짜 이전입니다. 키워드 탐색 종료.")
                        return results  # 루프 종료 후 수집된 결과 반환
                    
                    if article_date and article_date > end_date_str:
                        continue

                    results.append({
                        'media': '국민일보',
                        'title': title,
                        'date': date_str,
                        'section': '',
                        'article_type': '',
                        'writer': '',
                        'page': '',
                        'url': url,
                        'content': ''
                    })
            except Exception as e:
                print(f"기사 정보 추출 중 오류 발생: {e}")

        # 다음 페이지로 이동
        try:
            next_page_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, f'a[href*="pageNo={page + 1}"]'))
            )
            next_page_button.click()
            page += 1
            time.sleep(2)  # 페이지 로드 대기
        except Exception as e:
            print(f"페이지 탐색 중 오류 발생: {e}")
            break

    return results

def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df.to_excel(filename, index=False)

# 전체 결과 수집
all_results = []

for keyword in keywords:
    results = fetch_search_results(keyword, start_date, end_date)
    all_results.extend(results)

# 결과를 엑셀 파일로 저장
save_to_excel(all_results, f'1국민일보 {start_date}~{end_date}.xlsx')

# 드라이버 종료
driver.quit()


In [ ]:
# 본문 추출

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

def get_content_and_subtitle_from_url(url):
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 국민일보 기사 본문 추출
        article_body = soup.find('div', class_='article_body')
        subtitle_tag = soup.find('p', class_='article_subhead')

        subtitle = ""
        if subtitle_tag:
            subtitle = subtitle_tag.get_text(separator='\n', strip=True) if subtitle_tag else None
            subtitle_tag.decompose()  # 부제를 본문에서 제거

        if article_body:
            # 'figcaption' 태그를 가진 모든 캡션을 제거합니다.
            for caption in article_body.find_all('figcaption'):
                caption.decompose()

            # 본문에서 "GoodNews paper ⓒ" 이후 내용을 제거합니다.
            content = article_body.get_text(separator='\n', strip=True)
            cutoff_phrase = "GoodNews paper"
            if cutoff_phrase in content:
                content = content.split(cutoff_phrase)[0].strip()
        else:
            content = ""
        
        return content, subtitle
    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None, None

# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []  # 에러 로그 리스트 초기화
total_articles = len(df)  # 총 기사 수 계산

# 각 행의 URL에서 본문과 서브 타이틀을 스크랩하여 content와 sub-title 컬럼에 저장합니다.
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, subtitle = get_content_and_subtitle_from_url(row['url'])

    if content is None:
        error_log.append(row['url'])
    if content and content.startswith('='):
        content = "'" + content

    df.at[index, 'content'] = content
    df.at[index, 'sub-title'] = subtitle  # 서브 타이틀 컬럼에 저장

    # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 불필요한 항목을 제외한 후 엑셀 파일로 저장
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")

# 에러 로그 파일 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")
    print(f"에러 로그가 저장되었습니다: {error_log_path}")
